In [1]:
import numpy as np

In [2]:
import visa

In [3]:
rm = visa.ResourceManager()

In [7]:
print(rm.list_resources())

(u'USB0::0x0957::0x0618::MY52210065::INSTR', u'ASRL1::INSTR')


In [8]:
my_instrument = rm.open_resource('USB0::0x0957::0x0618::MY52210065::INSTR')

In [9]:
print(my_instrument.query('*IDN?'))

Agilent Technologies,34405A,MY52210065,1.47-3.13



In [11]:
print(my_instrument.query('CONF:VOLT:DC'))

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [14]:
for k in range(10):
    print(my_instrument.query('READ?'))

+1.39140000E-01 

+1.39260000E-01 

+1.39180000E-01 

+1.39320000E-01 

+1.39160000E-01 

+1.39280000E-01 

+1.39180000E-01 

+1.39310000E-01 

+1.39330000E-01 

+1.39240000E-01 



In [35]:
print(my_instrument.query('TRIG:SOUR?'))

IMM



In [24]:
print(my_instrument.query('MEAS:VOLT:DC?'))

+1.32340000E-01 



In [37]:
for k in range(10):
    print(my_instrument.query('FETCH?'))

+1.32110000E-01 

+1.32110000E-01 

+1.32110000E-01 

+1.32110000E-01 

+1.32110000E-01 

+1.32110000E-01 

+1.32110000E-01 

+1.32110000E-01 

+1.32110000E-01 

+1.32110000E-01 

